# Пакет Pandas
## Манипуляции над данными класса DataFrame, Series, Index

In [2]:
import numpy as np
import pandas as pd

### Иерархическая индексация или мультииндексация
Включение в один индекс нескольких уровней

In [3]:
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex([('California', 2000),
            ('California', 2010),
            (  'New York', 2000),
            (  'New York', 2010),
            (     'Texas', 2000),
            (     'Texas', 2010)],
           )

In [4]:
pop = pd.Series(populations, index=index)
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [4]:
# отбор по 2010 году
pop[:, 2010]

California    37253956
New York      19378102
Texas         25145561
dtype: int64

In [5]:
# преобразование в DataFrame
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [6]:
# обратно
pop_df.stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [7]:
pop_df = pd.DataFrame({'total': pop,
                       'under18': [9267089, 9284094,
                                   4687374, 4318033,
                                   5906301, 6879014]})
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

In [8]:
f_u18 = pop_df['under18'] / pop_df['total']
f_u18.unstack()

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


#### Методы создания мультииндексов
Наиболее простой метод создания мультииндексированного объекта Series или
DataFrame — передать в конструктор список из двух или более индексных массивов. 

In [9]:
df = pd.DataFrame(np.random.rand(4, 2),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=['data1', 'data2'])
df

data1     data2
a 1  0.217520  0.624089
  2  0.378609  0.517143
b 1  0.013155  0.084433
  2  0.895197  0.839052

Если передать словарь с соответствующими кортежами в качестве ключей, библиотека Pandas автоматически распознает такой синтаксис и будет по умолчанию использовать мультииндекс.

In [10]:
data = {('California', 2000): 33871648,
        ('California', 2010): 37253956,
        ('Texas', 2000): 20851820,
        ('Texas', 2010): 25145561,
        ('New York', 2000): 18976457,
        ('New York', 2010): 19378102}
pd.Series(data)

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
New York    2000    18976457
            2010    19378102
dtype: int64

#### Явные конструкторы для объектов MultiIndex

In [3]:
# из массивов
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [4]:
# из кортежей
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [5]:
# из декартового произведения обычных индексов
pd.MultiIndex.from_product([['a', 'b'], [1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

#### Названия уровней мультииндексов

In [6]:
pop.index.names = ['state', 'year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

#### Мультииндекс для столбцов

In [9]:
# Иерархические индексы и столбцы
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                    names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'],
                                      ['HR', 'Temp']],
                                     names=['subject', 'type'])
# Создаем имитационные данные
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37
# Создаем объект DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      55.0  37.5  41.0  37.4  41.0  36.4
     2      21.0  37.9  33.0  36.5  18.0  35.4
2014 1      44.0  37.2  29.0  36.6  27.0  37.9
     2      25.0  37.9  52.0  34.5  41.0  37.2

In [11]:
health_data['Guido']

type          HR  Temp
year visit            
2013 1      37.0  37.7
     2      43.0  37.7
2014 1      24.0  37.8
     2      50.0  38.8

#### Индексация и срезы по мультииндексу

In [12]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [13]:
pop['California', 2000]

33871648

In [14]:
pop.loc['California':'New York']

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

In [15]:
pop[:, 2000]

state
California    33871648
New York      18976457
Texas         20851820
dtype: int64

In [16]:
pop[pop > 22000000]

state       year
California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

In [17]:
health_data['Guido', 'HR']

year  visit
2013  1        37.0
      2        43.0
2014  1        24.0
      2        50.0
Name: (Guido, HR), dtype: float64

In [18]:
health_data.iloc[:2, :2]

subject      Bob      
type          HR  Temp
year visit            
2013 1      44.0  36.7
     2      35.0  35.9

In [19]:
health_data.loc[:, ('Bob', 'HR')]

year  visit
2013  1        44.0
      2        35.0
2014  1        26.0
      2        41.0
Name: (Bob, HR), dtype: float64

Можно избежать синтакс. ошибок со срезами, сформировав срез явным образом с помощью встроенной функции Python slice(), но лучше в данном случае использовать объект IndexSlice, предназначенный библиотекой Pandas как раз для подобной ситуации.

In [20]:
idx = pd.IndexSlice
health_data.loc[idx[:, 1], idx[:, 'HR']]

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,44.0,37.0,41.0
2014,1,26.0,24.0,21.0


#### Перегруппировка мультииндексов
#### Отсортированные и неотсортированные индексы
Большинство операций срезов с мультииндексами завершится ошибкой, если индекс не отсортирован (лексикографически упорядочен).

In [22]:
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ['char', 'int']
data

char  int
a     1      0.505060
      2      0.221278
c     1      0.185978
      2      0.596202
b     1      0.192952
      2      0.337006
dtype: float64

In [23]:
try:
    data['a':'b']
except KeyError as e:
    print(type(e))
    print(e)

<class 'pandas.errors.UnsortedIndexError'>
'Key length (1) was greater than MultiIndex lexsort depth (0)'


In [24]:
data = data.sort_index()
data

char  int
a     1      0.505060
      2      0.221278
b     1      0.192952
      2      0.337006
c     1      0.185978
      2      0.596202
dtype: float64

In [25]:
data['a':'b']

char  int
a     1      0.505060
      2      0.221278
b     1      0.192952
      2      0.337006
dtype: float64

#### Выполнение над индексами операций stack и unstack

In [26]:
pop.unstack(level=0)

state,California,New York,Texas
year,,,
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [27]:
pop.unstack(level=1)

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


#### Создание и перестройка индексов
Еще один способ перегруппировки иерархических данных — преобразовать метки индекса в столбцы с помощью метода reset_index.

In [7]:
pop_flat = pop.reset_index(name='population')
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


In [8]:
# обратно
pop_flat.set_index(['state', 'year'])

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

#### Агрегирование по мультииндексам
В библиотеке Pandas имеются встроенные методы для агрегирования данных, например mean(), sum() и max().

In [10]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      55.0  37.5  41.0  37.4  41.0  36.4
     2      21.0  37.9  33.0  36.5  18.0  35.4
2014 1      44.0  37.2  29.0  36.6  27.0  37.9
     2      25.0  37.9  52.0  34.5  41.0  37.2

In [13]:
data_mean = health_data.groupby(level='year').mean()
data_mean

subject   Bob        Guido          Sue       
type       HR   Temp    HR   Temp    HR   Temp
year                                          
2013     38.0  37.70  37.0  36.95  29.5  35.90
2014     34.5  37.55  40.5  35.55  34.0  37.55

In [16]:
# найти средний пульс и температуру по всем субъектам и визитам за каждый год
data_mean.groupby(level='type', axis=1).mean()

type,HR,Temp
year,,
2013,34.833333,36.850000
2014,36.333333,36.883333
